In [3]:
%cd ..

import math
import json
import utils
import os
import pandas as pd
import yfinance as yf
from itables import show
from multiprocessing import Pool 

if not os.path.exists('sp500.json'):
    sp500 = utils.spy_components()
    with open('./sp500.json', 'w') as f:
        json.dump(sp500, f, indent=2)
else:
    with open("./sp500.json", 'r') as f:
        sp500 = json.load(f)


/home/gangzhi/play_ground/stock_screener


In [4]:
pwd

'/home/gangzhi/play_ground/stock_screener'

In [5]:
import os
SAVE_DIR='./data'
file = 'sp500_daily_prices.pkl'
file_path = os.path.join(SAVE_DIR, file)

download = True
if os.path.exists(file_path): 
    answer = input("Re-download SP500? [yes/no]")
    download = True if answer[0].lower() == 'y' else False 
if download:
    daily_df = yf.download(sp500).swaplevel(axis=1)
    daily_df.to_pickle(file_path)
else:
    daily_df = pd.read_pickle(file_path)


Re-download SP500? [yes/no] no


In [ ]:
# Drop rows before the year 1990
daily_df = daily_df[daily_df.index.year >= 1990]
daily_df = daily_df[~(daily_df < 1).any(axis=1)]
split_daily_df = [daily_df[ticker] for ticker in sp500]


In [8]:
def feature_augment(df):
    # remove where volume is 0
    prev_close = df.Close.shift(1)
    prev_vol = df.Volume.shift(1)
    ret = pd.DataFrame()
    ret['forward_roc20'] = df.Close / df.Close.shift(-20)
    
    ret['high'] = df.High / prev_close
    ret['low'] = df.Low / prev_close
    ret['open'] = df.Open / prev_close
    ret['close'] = df.Close / prev_close
    ret['vol'] = df.Volume / prev_vol
    # cross features
    ret['vol_x_high'] = ret['vol'] * ret['high']
    ret['vol_x_low'] =  ret['vol'] * ret['low']
    ret['vol_x_open'] =  ret['vol'] * ret['open']
    ret['vol_x_close'] =  ret['vol'] * ret['close']
    ret.dropna(inplace=True)
    ret = ret.map(math.log)
    # Feature names: hloc, v, v crossed h,l.o,c
    return ret


In [14]:
%%time
with Pool(4) as pool:
    data = pool.map(feature_augment, split_daily_df)

CPU times: user 197 ms, sys: 262 ms, total: 459 ms
Wall time: 1.38 s
